# Oscillating cylinder

This notebook will analyze simulations of accelerating flow past an oscillating cylinder, using a no-slip or volume penalized inner boundary.

A python script is provided for parallelized simulation.

# Imports

In [ ]:
import numpy as np
import dedalus.public as de
import matplotlib.pyplot as plt
import os
from scipy.special import erf
import time
import logging
root = logging.root
for h in root.handlers: h.setLevel("INFO")
logger = logging.getLogger(__name__)

from dedalus.tools import post
from glob import glob
import h5py

# Functions

In [ ]:
def load_data(filename, *dnames, group='/',show=False,flatten=True,sel=None,checkint=True):
    """Load list of arrays given names of group in an hdf5 file.
    
    Parameters
    ----------
    dnames: list
        strings of dataset names
    filename: string
        file name
    group: string, optional
        subgroup of hdf5 file to write to
    overwrite: boolean, optional
    show: boolean, optional
    flatten: boolean, optional
        return number if single value
    sel: slice object, optional
        return slice of data array
    Returns
    -------
    List of numpy arrays

    """

    with h5py.File(filename,'r') as f:
        arrs = []
        g = f[group]
        for dname in dnames:
            if show: print(dname)    
            if not sel: sel = Ellipsis
            arr = g[dname][sel]
            if flatten:
                if arr.size == 1: arr = arr.item()
                elif np.prod(arr.shape) == max(arr.shape): arr = arr.flatten()                
                elif arr.shape[0] == 1: arr = arr[0,Ellipsis]
            if checkint and isinstance(arr,float) and arr.is_integer(): arr = int(arr)
            arrs.append(arr)
    return arrs

def get_keys(filename, group='/'):
    """ Helper to get keys of an hdf5 file/group."""
    with h5py.File(filename, 'r') as f:
        g = f[group]
        keys = sorted(list(g.keys()))
    return keys

# Plotting 2D Polar data
def extend_angle(*arrays):
    """Complete the periodic mesh to remove missing slice in polar pcolormesh."""
    return [np.concatenate([arr,arr[[0],:]],axis=0) for arr in arrays]

def polar_plot(θθ,rr,array,
               fig=None,ax=None,savename=False,dpi=200,colorbar=True,
               return_plot=False,wrap=True,**kwargs):
    """Wrapper to create a polar plot of a quantity."""
    if fig==None: fig, ax = plt.subplots(figsize=(4,6),subplot_kw=dict(projection='polar'))
    if wrap: θθ, rr, array = extend_angle(θθ,rr,array)
    plot = ax.pcolormesh(θθ,rr,array,**kwargs)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    if colorbar: plt.colorbar(plot,ax=ax,orientation='horizontal')
    if savename: plt.savefig(savename,dpi=dpi,bbox_inches='tight')
    if return_plot: return fig, ax, plot
    return fig, ax    

# Analysis and plotting

In [ ]:
# Merge simulation data
sim = 'cylinder-penalized'
data_dir, force_dir, params_dir = glob(os.path.join('runs',sim,'*'))

for folder in [data_dir, force_dir, params_dir]:
    post.merge_analysis(folder,cleanup=True)

In [ ]:
# Load simulation data
data_file = glob(os.path.join(data_dir,'*'))[0]
force_file = glob(os.path.join(force_dir,'*'))[0]
params_file = glob(os.path.join(params_dir,'*'))[0]

t, θ, r = load_data(data_file,'sim_time','θ/1.0','r/1.0',group='scales')
us,vs,ps,qs = load_data(data_file,'u','v','p','q',group='tasks')
Γ, = load_data(params_file,'Γ',group='tasks')

θθ = θ[:,None] + 0*r[None,:]
rr = 0*θ[:,None] + r[None,:]

In [ ]:
# Mask function
polar_plot(θθ,rr,Γ,cmap='Greys',vmin=0,vmax=1)

In [ ]:
# True vorticity
polar_plot(θθ,rr,qs[-1]/rr,cmap='PuOr',vmax=5,vmin=-5)

In [ ]:
# True pressure
polar_plot(θθ,rr,ps[-1] - 0.5*(us[-1]**2 + vs[-1]**2),
           cmap='viridis',vmin=-2,vmax=1)